In [1]:
from jira import JIRA
from dotenv import load_dotenv
import getpass
import os

load_dotenv()

JIRA_URL = os.environ.get("JIRA_URL")
EMAIL = os.environ.get("EMAIL")
API_TOKEN = os.environ.get("JIRA_API_TOKEN")

jira = JIRA(
    server=JIRA_URL,
    basic_auth=(EMAIL, API_TOKEN)
)

# Test connection
user = jira.current_user()
print(f"Connected as: {user}")

Connected as: 712020:4da4c4ca-c19f-47de-bbb6-3c4574de0abd


# Accès aux tickets

In [2]:
issues = jira.search_issues(
    'project = SMP ORDER BY created DESC',
    maxResults=5
)

for issue in issues:
    print(issue.key, issue.fields.summary)

SMP-8 Test issue from Python
SMP-7 Se connecter avec les bons identifiants
SMP-1 Connexion


In [6]:
for issue in issues:
    print(jira.issue(issue.key, fields="key,summary,content")) # ?

SMP-8
SMP-7
SMP-1


In [ ]:
issue = jira.issue("SMP-7")

print(issue.key)
print(issue.fields.summary)
print(issue.fields.status.name)

# Ajout de ticket

In [8]:
new_issue = jira.create_issue(project='SMP',
                              summary='New issue from jira-python',
                              description='Look into this one',
                              issuetype={'name': 'Story'})

Ou via dictionnaire:

In [11]:
issue_dict = {
    'project': 'SMP',
    'summary': 'New issue from jira-python',
    'description': 'Created via a dict',
    'issuetype': {'name': 'Story'},
}
new_issue = jira.create_issue(fields=issue_dict)

Ou plusieurs à la fois:

In [65]:
issue_list = [
    {
        'project': 'SMP',
        'summary': 'First issue of many',
        'description': 'Look into this one',
        'issuetype': {'name': 'Bug'},
    },
    {
        'project': {'key': 'SMP'},
        'summary': 'Second issue',
        'description': 'Another one',
        'issuetype': {'name': 'Bug'},
    },
    {
        'project': 'SMP',
        'summary': 'Last issue',
        'description': 'Final issue of batch.',
        'issuetype': {'name': 'Bug'},
    }]
issues = jira.create_issues(field_list=issue_list)

> Ne soulève pas d'exception en cas d'erreur!

In [19]:
for issue in issues:
    print(issue, end="\n\n")

{'status': 'Success', 'issue': <JIRA Issue: key='SMP-14', id='10105'>, 'error': None, 'input_fields': {'project': {'id': '10000'}, 'summary': 'First issue of many', 'description': 'Look into this one', 'issuetype': {'name': 'Bug'}}}

{'status': 'Success', 'issue': <JIRA Issue: key='SMP-15', id='10106'>, 'error': None, 'input_fields': {'project': {'key': 'SMP'}, 'summary': 'Second issue', 'description': 'Another one', 'issuetype': {'name': 'Bug'}}}

{'status': 'Success', 'issue': <JIRA Issue: key='SMP-16', id='10107'>, 'error': None, 'input_fields': {'project': {'id': '10000'}, 'summary': 'Last issue', 'description': 'Final issue of batch.', 'issuetype': {'name': 'Bug'}}}



# Modification de ticket

In [21]:
issue = jira.issue("SMP-13")

issue.update(summary='new summary', description='A new summary was added')
issue.update(assignee={'name': 'new_user'})    # reassigning in update requires issue edit permission

In [22]:
issue.update(fields={'summary': 'new summary', 'description': 'A new summary was added via a dict'})

In [23]:
issue.update(notify=False, description='A quiet description change was made')

# Suppression de ticket

In [24]:
issue.delete()

# JQL

In [35]:
# Search returns first 50 results, `maxResults` must be set to exceed this
issues_in_proj = jira.search_issues('project=SMP')
all_proj_issues_but_mine = jira.search_issues('project=SMP and assignee != currentUser()')

print(all_proj_issues_but_mine)

[]


In [38]:
# my top 5 issues due by the end of the week, ordered by priority
oh_crap = jira.search_issues('assignee = currentUser() and due < endOfWeek() order by priority desc', maxResults=5)

print(oh_crap)

[]


In [39]:
# Summaries of my last 3 reported issues
for issue in jira.search_issues('reporter = currentUser() order by created desc', maxResults=3):
    print('{}: {}'.format(issue.key, issue.fields.summary))

SMP-16: Last issue
SMP-14: First issue of many
SMP-13: Last issue


# Commentaires

In [44]:
issue = jira.issue("SMP-13")

comments_a = issue.fields.comment.comments
comments_b = jira.comments(issue) # comments_b == comments_a

print(comments_a, comments_b, sep="\n")

[<JIRA Comment: id='10276'>, <JIRA Comment: id='10277'>]
[<JIRA Comment: id='10276'>, <JIRA Comment: id='10277'>]


In [50]:
comment = jira.comment('SMP-13', '10276')
author = jira.comment('SMP-13', '10276').author.displayName
time = jira.comment('SMP-13', '10276').created

print(comment, author, time, sep="\n")

10276
Ménoua Khatchatrian
2026-01-28T14:34:38.774+0100


In [53]:
comment = jira.add_comment('SMP-13', 'abc')    # no Issue object required
comment = jira.add_comment(issue, 'new comment')  # for admins only

comment.update(body='updated comment body')
comment.update(body='updated comment body but no mail notification', notify=False)
comment.delete()

<Response [204]>

In [69]:
import re

issue = jira.issue('SMP-13')
regex_for_png = re.compile(r'\!(\S+?\.(jpg|png|bmp))\|?\S*?\!')
pngs_used_in_comment = regex_for_png.findall(issue.fields.comment.comments[0].body)
for attachment in issue.fields.attachment:
    if attachment.filename in pngs_used_in_comment:
        with open(attachment.filename, 'wb') as f:
            f.write(attachment.get())

print(attachment.filename)

expression_du_besoin_moon_match.png


# Projets

In [71]:
projects = jira.projects()

print(projects)

[<JIRA Project: key='MJ', name='Magic Jira', id='10066'>, <JIRA Project: key='SMP', name='Sample project', id='10000'>]


In [72]:
mj = jira.project("MJ")
print(mj.name)
print(mj.lead.displayName)

Magic Jira
Ménoua Khatchatrian


In [73]:
components = jira.project_components(mj)
[c.name for c in components]

jira.project_roles(mj)
versions = jira.project_versions(mj)
[v.name for v in reversed(versions)]

[]

# Pièces jointes

In [74]:
jira.add_attachment(issue=issue, attachment='../referentiel/moon_match/moon_match_original.pdf')

<JIRA Attachment: filename='moon_match_original.pdf', id='10034', mimeType='application/pdf'>

In [76]:
for attachment in issue.fields.attachment:
    print("Name: '{filename}', size: {size}".format(
        filename=attachment.filename, size=attachment.size))
    # to read content use `get` method:
    # print("Content: '{}'".format(attachment.get()))

Name: 'expression_du_besoin_moon_match.png', size: 153809
